In [1]:
import json
import os
from os import path
import time
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import spacy

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
                    'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

import mysql.connector

sql_config = {
    'user': 'root',
    'password': 'see-cret',
    'host': 'localhost',
    'database': 'tcc'
}


In [2]:
def extrai_dados_filename(filename):
    m = re.match('do_(?P<ano>\d{4})(?P<mes>\d{2})(?P<dia>\d{2})_(?P<edicao>\d{4})\.pdf\.xhtml\.json', filename)
    print(m.group('ano'), m.group('mes'), m.group('dia'), m.group('edicao'))

In [3]:
time_in = time.time()

extrator = re.compile('do_(?P<ano>\d{4})(?P<mes>\d{2})(?P<dia>\d{2})_(?P<edicao>\d{4})\.pdf\.xhtml\.json')

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    for filename in tqdm(os.listdir(DIRETORIO_PAR)):
        m = extrator.match(filename)
        
        m_ano = m.group('ano')
        m_mes = m.group('mes')
        m_dia = m.group('dia')
        m_edicao = m.group('edicao')
        
        try:
            cursor.execute('SELECT edicao FROM diario WHERE edicao = %s', (m_edicao,))
            cursor.fetchall()
            if cursor.rowcount == 0:
                cursor.execute('INSERT INTO diario(edicao, ano, mes, dia) VALUES (%s, %s, %s, %s)', (m_edicao, m_ano, m_mes, m_dia))
                cursor.fetchall()
            else:
                print('ERRO diario ja existente na tabela')
                cursor.close()
                connection.rollback()
                break
        except mysql.connector.Error as e:
            print('ERRO', e)
            cursor.close()
            connection.rollback()
            break
        
        with open(path.join(DIRETORIO_PAR, filename), 'r') as file:
            paragrafos = json.load(file)
        
        try:
            for num, paragrafo in enumerate(paragrafos):
                cursor.execute('INSERT INTO paragrafo (edicao, paragrafo, conteudo) VALUES (%s, %s, %s)', (m_edicao, num, paragrafo))
                cursor.fetchall()
        except mysql.connector.Error as e:
            print('ERRO', e)
            cursor.close()
            connection.rollback()
            break
            
    cursor.close()
    connection.commit()

time_out = time.time()
print(time_out - time_in)

100%|██████████| 950/950 [14:44<00:00,  1.07it/s]


903.0699677467346


In [39]:
with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo WHERE edicao = 3567')
    for c, in cursor:
        print(c)


ANO XXVII - Edição 3.567 www.bauru.sp.gov.br    SÁBADO, 25 DE JUNHO DE 2.022 EDIÇÃO DIGITAL

Diário Oficial de Bauru ASSINADO DIGITALMENTE 
PELO DIRETOR DO DEPARTAMENTO DE COMUNICAÇÃO E 

DOCUMENTAÇÃO

PODER EXECUTIVO
Suéllen Silva Rosim

Prefeita Municipal

Seção I
Gabinete da Prefeita

Rafael Lima Fernandes
Chefe de Gabinete

DECRETOS MUNICIPAIS
DECRETO Nº 16.152, DE 24 DE JUNHO DE 2.022

P. 65.560/2.020  Regulamenta a Lei Federal nº 14.314, de 24 de março de 2.022, que alterou a Lei 
complementar 173, de 27 de maio de 2.020, que ajusta o período de suspensão da contagem dos prazos de 
validade dos concursos públicos e processos seletivos em razão dos impactos econômicos decorrentes da 
emergência de saúde pública de importância internacional decorrente da Covid-19.

A PREFEITA MUNICIPAL DE BAURU, no uso de suas atribuições legais, 
conferidas pelo art. 51 da Lei Orgânica do Município de Bauru,
Considerando  que o art. 1º da Lei Federal nº 14.314, de 24 de março de 2.022, alterou o 

OSWALDO GOLZIO - AUTO DE INFRAÇÃO 10061537/2022 - PROCESSO 77205/2022

Aos vinte dias do mês de maio do ano de dois mil e vinte e dois, no endereço rua Bertolino Costa , quadra 02 
em Bauru Estado de São Paulo, foi constatado que OSWALDO GOLZIO, infringiu o disposto no Decreto 
Municipal 11689/2011, art. 45 “ Caberá aos geradores e transportadores o destino adequado dos resíduos 
da construção civil”.
Dando cumprimento ao Decreto Municipal 11.689/2011, artigo 56 - “Considera-se infração administrativa 
toda ação ou omissão praticada a título de dolo ou culpa que viole as disposições estabelecidas e ainda 
conforme definidas no Anexo III item VI alínea b:
“ Despejar resíduos da construção em locais proibidos ou não licenciados em veículos considerados leves, 
correspondendo a ciclomotor, motoneta, motocicleta, triciclo, quadrículo, automóvel , utilitário, camionete e 
camioneta lavro o presente AUTO DE INFRAÇÃO impondo-lhe a multa no valor de R$ 896,94 (oitocentos 
e noventa e seis reai